# APSRTC

In [1]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_apsrtc=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_apsrtc.append({'Route_link':route_link,'Route_name':route_name})
    return route_apsrtc

#Pagination code        
for page_number in range(1, 6):
    route_details()
    if page_number<5:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()
bus_details_text=[] 
route_links=[i['Route_link'] for i in route_apsrtc]

for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)


    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings="NA"
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price="NA"
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                         
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





In [4]:
APSRTC_route=pd.DataFrame(route_apsrtc)
APSRTC_bus_details=pd.DataFrame(bus_details_text)
APSRTC_bus=pd.merge(APSRTC_route,APSRTC_bus_details,on='Route_link')
APSRTC_bus_new=APSRTC_bus.dropna()




In [10]:
APSRTC_bus_new.to_csv("APSRTC_bus_details.csv")

In [7]:
APSRTC_bus_new

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 35119,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),09:30,06h 15m,15:45,4.2,720,42 Seats available\n20 Window
1,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 3543,Super Luxury (Non AC Seater 2+2 Push Back),10:15,07h 05m,17:20,4.0,469,29 Seats available\n13 Window
2,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 9310,Super Luxury (Non AC Seater 2+2 Push Back),10:15,06h 35m,16:50,3.9,436,33 Seats available\n16 Window
3,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 3657,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),10:30,05h 55m,16:25,3.9,720,45 Seats available\n22 Window
4,https://www.redbus.in/bus-tickets/vijayawada-t...,Vijayawada to Hyderabad,APSRTC - 3513,Super Luxury (Non AC Seater 2+2 Push Back),10:39,09h 06m,19:45,2.1,488,33 Seats available\n16 Window
...,...,...,...,...,...,...,...,...,...,...
9732,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Hyderabad,KSM Roadways,Bharat Benz A/C Sleeper (2+1),23:55,05h 35m,05:30,4.7,1899,36 Seats available
9733,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Hyderabad,Al madeena Travels,A/C Sleeper (2+1),22:15,08h 12m,06:27,3.9,950,21 Seats available\n7 Single
9734,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Hyderabad,SRI SAI MURALI KRISHNA TRAVELS,A/C Sleeper (2+1),23:30,07h 15m,06:45,3.2,1245,30 Seats available\n10 Single
9735,https://www.redbus.in/bus-tickets/ananthapur-t...,Anantapur (andhra pradesh) to Hyderabad,Bmcc Travels,NON A/C Sleeper (2+1),23:59,05h 31m,05:30,3.0,952,24 Seats available


In [18]:
APSRTC_bus_new['Rating']=APSRTC_bus_new['Rating'].replace('NA','0.0')

In [ ]:
APSRTC_bus_new['Rating']=APSRTC_bus_new['Rating'].replace('NA','0.0')

In [5]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
#mycursor.execute("CREATE DATABASE RedBus_project_new")


In [19]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS APSRTC_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO APSRTC_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in APSRTC_bus_new.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# KSRTC

In [8]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_ksrtc=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_ksrtc.append({'Route_link:':route_link,'Route_name:':route_name})
    return route_ksrtc

#Pagination code        
for page_number in range(1, 3):
    route_details()
    if page_number<2:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link:'] for i in route_ksrtc]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings="NA"
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price="NA"
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link:':link,'Bus_name:':bus_name,'Bus_type:':bus_type,'Dep_time:':departure_time,'Duration:':duration,'Arr_time:':arrival_time,'Rating:':ratings,'Price:':price,'Seat_avail:':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





In [19]:

KSRTC_route=pd.DataFrame(route_ksrtc)
KSRTC_bus_details=pd.DataFrame(bus_details_text)
KSRTC_bus=pd.merge(KSRTC_route,KSRTC_bus_details,on='Route_link:')
KSRTC_bus_new=KSRTC_bus.dropna()
KSRTC_bus_new

,Route_link:,Route_name:,Bus_name:,Bus_type:,Dep_time:,Duration:,Arr_time:,Rating:,Price:,Seat_avail:
0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,INR 627,34 Seats available
1,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,4.3,INR 620,13 Seats available\n4 Window
2,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.2,INR 729,38 Seats available\n15 Window
3,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:02,08h 48m,00:50,2.8,INR 567,23 Seats available\n7 Window
4,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,09h 20m,04:24,4.1,INR 640,38 Seats available\n17 Window
...,...,...,...,...,...,...,...,...,...,...
1438,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Sonia Travels,A/C Seater/Sleeper (2+1),19:30,03h 59m,23:29,3.8,INR 750,20 Seats available
1439,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,A/C Sleeper (2+1),22:30,01h 29m,23:59,4.0,INR 850,17 Seats available\n5 Single
1440,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),22:25,01h 34m,23:59,4.0,INR 667,29 Seats available\n11 Window
1441,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,A/C Sleeper (2+1),22:30,01h 29m,23:59,4.0,INR 850,17 Seats available\n5 Single


In [26]:
KSRTC_bus_new['Price:']=KSRTC_bus_new['Price:'].str.replace('INR','').astype(float)

In [27]:
KSRTC_bus_new

,Route_link:,Route_name:,Bus_name:,Bus_type:,Dep_time:,Duration:,Arr_time:,Rating:,Price:,Seat_avail:
0,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 129,SWIFT-GARUDA A/C SEATER BUS,14:15,08h 35m,22:50,4.2,627.0,34 Seats available
1,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 235,Swift Deluxe Non AC Air Bus (2+2),15:00,09h 01m,00:01,4.3,620.0,13 Seats available\n4 Window
2,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 370,AC MULTI AXLE,15:25,06h 55m,22:20,2.2,729.0,38 Seats available\n15 Window
3,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 97,Swift Deluxe Non AC Air Bus (2+2),16:02,08h 48m,00:50,2.8,567.0,23 Seats available\n7 Window
4,https://www.redbus.in/bus-tickets/bangalore-to...,Bangalore to Kozhikode,KSRTC (Kerala) - 51,Swift Deluxe Non AC Air Bus (2+2),19:04,09h 20m,04:24,4.1,640.0,38 Seats available\n17 Window
...,...,...,...,...,...,...,...,...,...,...
1438,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Sonia Travels,A/C Seater/Sleeper (2+1),19:30,03h 59m,23:29,3.8,750.0,20 Seats available
1439,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,A/C Sleeper (2+1),22:30,01h 29m,23:59,4.0,850.0,17 Seats available\n5 Single
1440,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),22:25,01h 34m,23:59,4.0,667.0,29 Seats available\n11 Window
1441,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kalpetta (kerala) to Kozhikode,Cochin Express,A/C Sleeper (2+1),22:30,01h 29m,23:59,4.0,850.0,17 Seats available\n5 Single


In [53]:
KSRTC_bus_new.rename(columns={'Route_link:':'Route_link','Route_name:':'Route_name','Bus_name:':'Bus_name','Bus_type:':'Bus_type','Dep_time:':'Dep_time','Duration:':'Duration','Arr_time:':'Arr_time','Rating:':'Rating','Price:':'Price','Seat_avail:':'Seat_avail'}, inplace=True)

In [28]:
KSRTC_bus_new.to_csv("Kerala_bus_details.csv")

In [62]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor.execute("CREATE DATABASE RedBus_project_new")


1

In [63]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS KSRTC_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)
# try:
#     mycursor.execute(create_table_query)
# except pymysql.MySQLError as err:
#     print("--")
# con.commit()

data="INSERT INTO KSRTC_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in KSRTC_bus_new.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# TSRTC

In [2]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_tsrtc=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_tsrtc.append({'Route_link':route_link,'Route_name':route_name})
    return route_tsrtc

#Pagination code        
for page_number in range(1, 4):
    route_details()
    if page_number<3:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [11]:
route_tsrtc

[{'Route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
  'Route_name': 'Khammam to Hyderabad'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-vijayawada',
  'Route_name': 'Hyderabad to Vijayawada'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-khammam',
  'Route_name': 'Hyderabad to Khammam'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-srisailam',
  'Route_name': 'Hyderabad to Srisailam'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/karimnagar-to-hyderabad',
  'Route_name': 'Karimnagar to Hyderabad'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-nirmal',
  'Route_name': 'Hyderabad to Nirmal'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-mancherial',
  'Route_name': 'Hyderabad to Mancherial'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderabad-to-adilabad',
  'Route_name': 'Hyderabad to Adilabad'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/hyderab

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_tsrtc]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings="NA"
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price="NA"
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





In [4]:
bus_details_text

[{'Route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
  'Bus_name': 'TSRTC - 8824',
  'Bus_type': 'SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)',
  'Dep_time': '11:50',
  'Duration': '04h 40m',
  'Arr_time': '16:30',
  'Rating': '3.9',
  'Price': '320',
  'Seat_avail': '25 Seats available'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
  'Bus_name': 'TSRTC - 8609',
  'Bus_type': 'RAJDHANI (A.C. Semi Sleeper)',
  'Dep_time': '12:10',
  'Duration': '04h 00m',
  'Arr_time': '16:10',
  'Rating': '3.3',
  'Price': '389',
  'Seat_avail': '23 Seats available\n9 Window'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
  'Bus_name': 'TSRTC - 8918',
  'Bus_type': '',
  'Dep_time': '12:30',
  'Duration': '04h 20m',
  'Arr_time': '16:50',
  'Rating': '3.5',
  'Price': '320',
  'Seat_avail': '17 Seats available'},
 {'Route_link': 'https://www.redbus.in/bus-tickets/khammam-to-hyderabad',
  'Bus_name': '',
  'Bus_type': '',
  'Dep_

In [5]:
import pandas as pd
TSRTC_route=pd.DataFrame(route_tsrtc)
TSRTC_bus_details=pd.DataFrame(bus_details_text)
TSRTC_bus=pd.merge(TSRTC_route,TSRTC_bus_details,on='Route_link')
TSRTC_bus_new=TSRTC_bus.dropna(inplace=True)
TSRTC_bus_new

In [10]:
TSRTC_bus

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8824,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:50,04h 40m,16:30,3.9,320,25 Seats available
1,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8609,RAJDHANI (A.C. Semi Sleeper),12:10,04h 00m,16:10,3.3,389,23 Seats available\n9 Window
2,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8918,,12:30,04h 20m,16:50,3.5,320,17 Seats available
3,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,,,,,,,,
4,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
688,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,
689,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,NA,
690,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,
691,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,


In [11]:
TSRTC_bus.dropna()

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8824,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:50,04h 40m,16:30,3.9,320,25 Seats available
1,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8609,RAJDHANI (A.C. Semi Sleeper),12:10,04h 00m,16:10,3.3,389,23 Seats available\n9 Window
2,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8918,,12:30,04h 20m,16:50,3.5,320,17 Seats available
3,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,,,,,,,,
4,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
688,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,
689,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,NA,
690,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,
691,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,,,,,,,,


In [23]:
TSRTC_bus_new=pd.read_csv(r'H:\IIT-M GUVI\Projects\Project 1\New folder\TSRTC_bus_new.csv')

In [24]:
TSRTC_bus_new=TSRTC_bus_new.rename(columns={'Route_link:':'Route_link','Route_name:':'Route_name','Bus_name:':'Bus_name','Bus_type:':'Bus_type','Dep_time:':'Dep_time','Duration:':'Duration','Arr_time:':'Arr_time','Rating:':'Rating','Price:':'Price','Seat_avail:':'Seat_avail'})

In [25]:
TSRTC_bus_new

,Unnamed: 0,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,0,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 64824,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:30,04h 20m,14:50,2.5,320,30 Seats available
1,1,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8968,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:35,04h 35m,15:10,3.3,320,25 Seats available
2,2,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8726,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",10:40,04h 30m,15:10,2.1,320,29 Seats available
3,3,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 8847,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",11:10,04h 20m,15:30,4.1,320,31 Seats available
4,4,https://www.redbus.in/bus-tickets/khammam-to-h...,Khammam to Hyderabad,TSRTC - 34339,RAJDHANI (A.C. Semi Sleeper),11:20,04h 10m,15:30,2.6,389,35 Seats available\n17 Window
...,...,...,...,...,...,...,...,...,...,...,...
4096,4096,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,SREE LEKYAA TRAVELS,NON A/C Sleeper (2+1),21:40,11h 05m,08:45,3.7,1000,13 Seats available
4097,4097,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,Sri Krishna Travels,A/C Sleeper (2+1),17:45,14h 45m,08:30,3.6,2190,30 Seats available\n10 Single
4098,4098,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,RK Travels,Non A/C Seater / Sleeper (2+1),21:00,0-12h 0-30m,09:30,3.5,5000,36 Seats available\n25 Window
4099,4099,https://www.redbus.in/bus-tickets/hyderabad-to...,Hyderabad to Tirupati,GOWTHAMI TOURS AND TRAVELS,A/C Sleeper (2+1),22:00,11h 00m,09:00,3.2,1520,26 Seats available


In [21]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor
#mycursor.execute("CREATE DATABASE RedBus_project_new")

In [26]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS TSRTC_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO TSRTC_bus_details(Route_link, Route_name, Bus_name, Bus_type, Dep_time, Duration, Arr_time, Rating, Price, Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in TSRTC_bus_new.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# KTCL

In [85]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_ktcl=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_ktcl.append({'Route_link':route_link,'Route_name':route_name})
    return route_ktcl

#Pagination code        
for page_number in range(1, 5):
    route_details()
    if page_number<4:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [86]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_ktcl]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings=0
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price=0
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA


In [87]:
KTCL_route=pd.DataFrame(route_ktcl)
KTCL_bus_details=pd.DataFrame(bus_details_text)



In [88]:
KTCL_bus=pd.merge(KTCL_route,KTCL_bus_details,on='Route_link')

In [89]:
KTCL_bus['Price']=KTCL_bus['Price'].replace('NA',0)

In [90]:
KTCL_bus['Price']=KTCL_bus['Price'].replace('',0)

In [91]:
KTCL_bus['Rating']=KTCL_bus['Rating'].replace('NA',0)

In [92]:
KTCL_bus['Rating']=KTCL_bus['Rating'].replace('',0)

In [96]:
KTCL_bus.to_csv("KTCL_bus_details.csv")

In [94]:
KTCL_bus.dropna()

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.6,750,1 Seat available\n1 Single
1,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa,Ashray Vip Yellows,A/C Sleeper (2+1),20:30,11h 30m,08:00,4.4,700,20 Seats available
2,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa,Ashray Travels,Bharat Benz A/C Sleeper (2+1),21:00,11h 30m,08:30,4.4,649,24 Seats available
3,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa,Zingbus Plus,Bharat Benz A/C Sleeper (2+1),23:00,11h 00m,10:00,4.4,0,21 Seats available\n6 Single
4,https://www.redbus.in/bus-tickets/pune-to-goa,Pune to Goa,,,,,,0,0,
...,...,...,...,...,...,...,...,...,...,...
797,https://www.redbus.in/bus-tickets/goa-airport-...,Goa Airport to Panaji,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),16:00,01h 00m,17:00,4.4,150,42 Seats available\n20 Window
798,https://www.redbus.in/bus-tickets/goa-airport-...,Goa Airport to Panaji,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),13:45,01h 00m,14:45,4.4,150,41 Seats available\n19 Window
799,https://www.redbus.in/bus-tickets/goa-airport-...,Goa Airport to Panaji,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),16:00,01h 00m,17:00,4.4,150,42 Seats available\n20 Window
800,https://www.redbus.in/bus-tickets/goa-airport-...,Goa Airport to Panaji,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),13:45,01h 00m,14:45,4.4,150,41 Seats available\n19 Window


In [99]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor


In [100]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS KTCL_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO KTCL_bus_details(Route_link, Route_name, Bus_name, Bus_type, Dep_time, Duration, Arr_time, Rating, Price, Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in KTCL_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# RSRTC

In [117]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_rsrtc=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_rsrtc.append({'Route_link':route_link,'Route_name':route_name})
    return route_rsrtc

#Pagination code        
for page_number in range(1, 4):
    route_details()
    if page_number<3:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [118]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_rsrtc]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings="NA"
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price="NA"
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





In [120]:
RSRTC_route=pd.DataFrame(route_rsrtc)
RSRTC_bus_details=pd.DataFrame(bus_details_text)
RSRTC_bus=pd.merge(RSRTC_route,RSRTC_bus_details,on='Route_link')

In [121]:
RSRTC_bus_new=RSRTC_bus.dropna()

In [125]:
RSRTC_bus_new['Rating']=RSRTC_bus_new['Rating'].replace('NA','0')

In [129]:
RSRTC_bus_new['Price']=RSRTC_bus_new['Price'].replace('NA','0')

In [130]:
RSRTC_bus_new.to_csv("RSRTC_bus_details.csv")

In [131]:
RSRTC_bus_new

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 160781,Super Luxury Volvo AC Seater Pushback 2+2,00:00,14h 00m,14:00,4.6,460,37 Seats available\n18 Window
1,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 192566,Semi Deluxe Non AC Seater 2+2,04:45,04h 30m,09:15,4.2,250,35 Seats available\n17 Window
2,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 146314,Super Luxury Volvo AC Seater Pushback 2+2,06:15,04h 00m,10:15,4.3,460,23 Seats available\n9 Window
3,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 193343,Semi Deluxe Non AC Seater 2+2,07:15,04h 45m,12:00,4.2,250,35 Seats available\n17 Window
4,https://www.redbus.in/bus-tickets/jodhpur-to-a...,Jodhpur to Ajmer,RSRTC - 185141,Super Luxury Volvo AC Seater Pushback 2+2,07:15,04h 00m,11:15,4.2,460,37 Seats available\n18 Window
...,...,...,...,...,...,...,...,...,...,...
3526,https://www.redbus.in/bus-tickets/bikaner-to-s...,Bikaner to Sikar,Amar Travels(Jaipur),AC Sleeper (2+1),22:00,04h 45m,02:45,2.5,500,34 Seats available
3527,https://www.redbus.in/bus-tickets/bikaner-to-s...,Bikaner to Sikar,Sharma travels,A/C Seater / Sleeper (2+1),22:30,04h 30m,03:00,2.4,429,34 Seats available\n22 Window
3528,https://www.redbus.in/bus-tickets/bikaner-to-s...,Bikaner to Sikar,Amar Travels(Jaipur),A/C Seater / Sleeper (2+1),22:45,04h 45m,03:30,2.5,400,44 Seats available\n12 Single
3529,https://www.redbus.in/bus-tickets/bikaner-to-s...,Bikaner to Sikar,M R Travels,Volvo Multi-Axle A/C seater/sleeper (2+1),15:15,03h 00m,18:15,3.8,1200,60 Seats available


In [119]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [132]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS RSRTC_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO RSRTC_bus_details(Route_link, Route_name, Bus_name, Bus_type, Dep_time, Duration, Arr_time, Rating, Price, Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in RSRTC_bus_new.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# Chandigarh

In [20]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_chandigarh=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_chandigarh.append({'Route_link':route_link,'Route_name':route_name})
    return route_chandigarh

#Pagination code        
for page_number in range(1, 6):
    route_details()
    if page_number<5:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_chandigarh]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings="NA"
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price="NA"
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





NA
NA
NA


In [22]:
Chandigarh_route=pd.DataFrame(route_chandigarh)
Chandigarh_bus_details=pd.DataFrame(bus_details_text)


In [23]:
Chandigarh_bus=pd.merge(Chandigarh_route,Chandigarh_bus_details,on='Route_link')
Chandigarh_bus

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165689,HVAC Seater (2+3),10:30,05h 35m,16:05,3.6,404.76,46 Seats available\n17 Window
1,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165691,HVAC Seater (2+3),11:20,05h 35m,16:55,3.9,404.76,41 Seats available\n15 Window
2,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165693,HVAC Seater (2+3),12:30,05h 35m,18:05,3.8,404.76,44 Seats available\n17 Window
3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165697,HVAC Seater (2+3),14:00,05h 35m,19:35,4.1,404.76,47 Seats available\n18 Window
4,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165709,HVAC Seater (2+3),14:30,05h 35m,20:05,3.9,404.76,46 Seats available\n17 Window
...,...,...,...,...,...,...,...,...,...,...
2358,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,Asian's Shina Tour & Travels (Swoyambhu Manjus...,NON A/C Semi Sleeper (2+2),23:55,05h 05m,05:00,2.5,999,29 Seats available\n14 Window
2359,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1564,Ordinary 3+2 Non AC Seater,12:15,05h 15m,17:30,3.9,300,27 Seats available\n9 Window
2360,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1514,Ordinary 3+2 Non AC Seater,16:00,05h 45m,21:45,3.9,318,37 Seats available\n13 Window
2361,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1285,Ordinary 3+2 Non AC Seater,18:40,05h 20m,00:00,3.9,293,37 Seats available\n13 Window


In [24]:
Chandigarh_bus.to_csv('Chandigarh_bus_details.csv')

In [25]:
Chandigarh_bus.dropna(inplace=True)

In [26]:
Chandigarh_bus['Price']=Chandigarh_bus['Price'].replace('NA',0)

In [29]:
Chandigarh_bus['Rating']=Chandigarh_bus['Rating'].replace('NA',0)

In [30]:
Chandigarh_bus['Price']=Chandigarh_bus['Price'].replace('',0)

In [31]:
Chandigarh_bus['Rating']=Chandigarh_bus['Rating'].replace('',0)

In [33]:
Chandigarh_bus_new=Chandigarh_bus.to_csv('Chandigarh_bus_details_new.csv')

In [37]:
Chandigarh_bus

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165689,HVAC Seater (2+3),10:30,05h 35m,16:05,3.6,404.76,46 Seats available\n17 Window
1,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165691,HVAC Seater (2+3),11:20,05h 35m,16:55,3.9,404.76,41 Seats available\n15 Window
2,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165693,HVAC Seater (2+3),12:30,05h 35m,18:05,3.8,404.76,44 Seats available\n17 Window
3,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165697,HVAC Seater (2+3),14:00,05h 35m,19:35,4.1,404.76,47 Seats available\n18 Window
4,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Delhi,Chandigarh Transport Undertaking (CTU) - 165709,HVAC Seater (2+3),14:30,05h 35m,20:05,3.9,404.76,46 Seats available\n17 Window
...,...,...,...,...,...,...,...,...,...,...
2358,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,Asian's Shina Tour & Travels (Swoyambhu Manjus...,NON A/C Semi Sleeper (2+2),23:55,05h 05m,05:00,2.5,999,29 Seats available\n14 Window
2359,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1564,Ordinary 3+2 Non AC Seater,12:15,05h 15m,17:30,3.9,300,27 Seats available\n9 Window
2360,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1514,Ordinary 3+2 Non AC Seater,16:00,05h 45m,21:45,3.9,318,37 Seats available\n13 Window
2361,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Jawala Ji,HRTC - 1285,Ordinary 3+2 Non AC Seater,18:40,05h 20m,00:00,3.9,293,37 Seats available\n13 Window


In [34]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [38]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS Chandigarh_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""
mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO Chandigarh_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in Chandigarh_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# Punjab

In [39]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_punjab=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_punjab.append({'Route_link':route_link,'Route_name':route_name})
    return route_punjab

#Pagination code        
for page_number in range(1, 4):
    route_details()
    if page_number<3:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_punjab]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings=0
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price=0
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





In [41]:
Punjab_route=pd.DataFrame(route_punjab)
Punjab_bus_details=pd.DataFrame(bus_details_text)

In [42]:
Punjab_bus=pd.merge(Punjab_route,Punjab_bus_details,on='Route_link')

In [43]:
Punjab_bus.to_csv('Punjab_bus_details.csv')

In [44]:
Punjab_bus.dropna()

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi,PEPSU (Punjab) - 66649,Volvo AC Seater (2+2),12:40,05h 25m,18:05,4.3,692,21 Seats available\n4 Window
1,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi,PEPSU (Punjab) - 67101,AC Seater Hvac 2+2,15:25,05h 01m,20:26,3.3,440,40 Seats available\n19 Window
2,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi,PEPSU (Punjab) - 67142,Volvo AC Seater (2+2),16:00,05h 25m,21:25,4.0,692,37 Seats available\n18 Window
3,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi,PEPSU (Punjab) - 66687,Volvo AC Seater (2+2),18:10,05h 25m,23:35,3.8,692,37 Seats available\n17 Window
4,https://www.redbus.in/bus-tickets/patiala-to-d...,Patiala to Delhi,PEPSU (Punjab) - 67104,AC Seater Hvac 2+2,22:30,05h 01m,03:31,2.8,440,40 Seats available\n19 Window
...,...,...,...,...,...,...,...,...,...,...
1257,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Patiala,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),13:30,01h 50m,15:20,2.3,200,45 Seats available\n22 Window
1258,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Patiala,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),13:55,01h 45m,15:40,2.5,200,45 Seats available\n22 Window
1259,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Patiala,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),15:25,01h 20m,16:45,2.2,200,47 Seats available\n22 Window
1260,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh to Patiala,Orbit Aviation Pvt. Ltd.,A/C Seater (2+2),16:55,01h 35m,18:30,2.7,200,44 Seats available\n21 Window


In [47]:
Punjab_bus['Rating']=Punjab_bus['Rating'].replace('',0)

In [48]:
Punjab_bus['Price']=Punjab_bus['Price'].replace('',0)

In [45]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [49]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS Punjab_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""

mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO Punjab_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in Punjab_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# BSRTC

In [50]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_bsrtc=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_bsrtc.append({'Route_link':route_link,'Route_name':route_name})
    return route_bsrtc

#Pagination code        
for page_number in range(1, 5):
    route_details()
    if page_number<4:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_bsrtc]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings=0
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price=0
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA


In [52]:
Bihar_route=pd.DataFrame(route_bsrtc)
Bihar_bus_details=pd.DataFrame(bus_details_text)


In [53]:
Bihar_bus_details.dropna(inplace=True)

In [54]:
Bihar_bus=pd.merge(Bihar_route,Bihar_bus_details,on='Route_link')

In [55]:
Bihar_bus

,Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Om Sai Ram (C.T),NON A/C Seater (2+2),14:10,06h 50m,21:00,3.3,500,28 Seats available\n14 Window
1,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Om Sai Ram (C.T),NON A/C Seater (2+2),14:50,07h 00m,21:50,3.3,450,28 Seats available\n14 Window
2,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,A/C Seater / Sleeper (2+2),20:28,05h 32m,02:00,3.4,0,6 Seats available\n3 Window
3,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.5,550,52 Seats available\n24 Window
4,https://www.redbus.in/bus-tickets/patna-to-bet...,Patna (Bihar) to Bettiah,Jai Mata Di Tour and Travels,,22:40,05h 20m,04:00,3.2,0,
...,...,...,...,...,...,...,...,...,...,...
673,https://www.redbus.in/bus-tickets/lucknow-to-g...,Lucknow to Gopalganj (Bihar),Gorakhdham Express Bus Service,A/C Seater / Sleeper (2+2),23:55,07h 35m,07:30,1.6,799,30 Seats available\n15 Window
674,https://www.redbus.in/bus-tickets/lucknow-to-g...,Lucknow to Gopalganj (Bihar),Panwar Travels,A/C Seater / Sleeper (2+1),21:30,06h 35m,04:05,2.8,2500,38 Seats available\n5 Single
675,https://www.redbus.in/bus-tickets/lucknow-to-g...,Lucknow to Gopalganj (Bihar),Maan Travels,NON AC Seater / Sleeper 2+1,23:55,10h 35m,10:30,1.6,950,25 Seats available\n5 Single
676,https://www.redbus.in/bus-tickets/lucknow-to-g...,Lucknow to Gopalganj (Bihar),Maa Laxmi Bihar Express,Non A/C Seater / Sleeper (2+1),22:30,09h 30m,08:00,1.3,1000,28 Seats available\n14 Window


In [57]:
Bihar_bus['Rating']=Bihar_bus['Rating'].replace('',0)

In [58]:
Bihar_bus['Price']=Bihar_bus['Price'].replace('',0)

In [59]:
Bihar_bus.to_csv('Bihar_bus_details.csv')

In [60]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [61]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS Bihar_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""
mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO Bihar_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in Bihar_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# West Bengal

In [62]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/west-bengal-transport-corporation?utm_source=rtchometile"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_wb=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_wb.append({'Route_link':route_link,'Route_name':route_name})
    return route_wb
route_details()
#Pagination code        
# for page_number in range(1, 3):
#     route_details()
#     if page_number<2:
        
#         try:
#             time.sleep(3)
#             pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
#             page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
#             actions=ActionChains(driver)
#             actions.move_to_element(page_link).perform()
#             time.sleep(3)
#             page_link.click()
#             wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
#             #route_details()            
            
#         except Exception as e:
#              #print("--")
#              break
        
    
wait=WebDriverWait(driver, 20) 
        
time.sleep(20)  

In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_wb]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings=0
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price=0
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





NA
NA
NA


In [64]:
WB_route=pd.DataFrame(route_wb)
WB_bus_details=pd.DataFrame(bus_details_text)

In [65]:
WB_bus_details.dropna()

,Route_link,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/kolkata-to-d...,WBTC (CTC) Belgachia - Digha via Esplanade - 5...,Non AC Seater (2+3),14:00,04h 15m,18:15,3.4,145,48 Seats available\n17 Window
1,https://www.redbus.in/bus-tickets/kolkata-to-d...,WBTC (CTC) Kolkata - Digha - 124|14:40,Non AC Seater (2+3),14:40,04h 25m,19:05,3.8,145,49 Seats available\n18 Window
2,https://www.redbus.in/bus-tickets/kolkata-to-d...,WBTC (CTC) Belgachia - Digha via Esplanade - 5...,Non AC Seater (2+3),14:45,05h 25m,20:10,3.4,155,48 Seats available\n17 Window
3,https://www.redbus.in/bus-tickets/kolkata-to-d...,WBTC (CTC) Belgachia - Digha via Esplanade - 5...,Non AC Seater (2+3),15:45,04h 25m,20:10,3.1,145,48 Seats available\n17 Window
4,https://www.redbus.in/bus-tickets/kolkata-to-d...,WBTC (CTC) Khidderpur - Digha - 97|16:00,Non AC Seater (2+3),16:00,05h 00m,21:00,3.4,151,49 Seats available\n18 Window
...,...,...,...,...,...,...,...,...,...
649,https://www.redbus.in/bus-tickets/kolkata-to-m...,Ankita Paribahan,AC Seater (2+3),13:50,04h 30m,18:20,2.2,0,41 Seats available\n16 Window
650,https://www.redbus.in/bus-tickets/kolkata-to-m...,Ankita Paribahan(Atithi),A/C Seater Push Back (2+3),22:40,05h 50m,04:30,2.2,0,20 Seats available\n8 Window
651,https://www.redbus.in/bus-tickets/kolkata-to-m...,Ankita Paribahan(Parameswar),A/C Seater (2+3),21:50,04h 10m,02:00,2.2,0,15 Seats available\n6 Window
652,https://www.redbus.in/bus-tickets/kolkata-to-m...,Suvam Travels,Non AC Seater (2+3),16:30,03h 50m,20:20,1.8,200,59 Seats available\n23 Window


In [66]:
WB_bus=pd.merge(WB_route,WB_bus_details,on='Route_link')

In [68]:
WB_bus['Price']=WB_bus['Price'].replace('',0)

In [69]:
WB_bus['Rating']=WB_bus['Rating'].replace('',0)

In [70]:
WB_bus.to_csv('WB_bus_details.csv')

In [71]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [73]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS WB_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""
mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO WB_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in WB_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()





# J&K

In [74]:


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import TimeoutException
from selenium.webdriver import ActionChains

driver = webdriver.Chrome()

url = "https://www.redbus.in/online-booking/jksrtc"
driver.get(url)

body = driver.find_element(By.TAG_NAME, 'body')
body.send_keys(Keys.PAGE_DOWN)

wait=WebDriverWait(driver, 10)  

#empty list to get route names and links 
route_jk=[]
#function to add details to route list
def route_details():
    
    route_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='route_link']")))
    time.sleep(5)
    for i in route_container:
        route_name=i.find_element(By.XPATH,".//a[@class='route']").text 
        route_link = i.find_element(By.XPATH,".//a").get_attribute('href') 
        route_jk.append({'Route_link':route_link,'Route_name':route_name})
    return route_jk

#Pagination code        
for page_number in range(1, 3):
    route_details()
    if page_number<2:
        
        try:
            time.sleep(3)
            pagination_container = wait.until(EC.presence_of_element_located((By.XPATH,"//div[@class='DC_117_paginationTable']")))
            page_link = pagination_container.find_element(By.XPATH,f'.//div[contains(@class,"DC_117_pageTabs ") and text()="{page_number+1}"]')
            actions=ActionChains(driver)
            actions.move_to_element(page_link).perform()
            time.sleep(3)
            page_link.click()
            wait.until(EC.text_to_be_present_in_element((By.XPATH,'//div[contains(@class,"DC_117_pageTabs DC_117_pageActive")]'), str(page_number + 1)))
            #route_details()            
            
        except Exception as e:
             #print("--")
             break
        
    
    wait=WebDriverWait(driver, 20) 
        
    time.sleep(20)  

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.common.exceptions import TimeoutException
import pandas as pd

driver = webdriver.Chrome()

bus_details_text=[] 
route_links=[i['Route_link'] for i in route_jk]
for link in route_links:
    driver.get(link)

    time.sleep(5)
    wait=WebDriverWait(driver,10)
    
    def bus_details():
        
        try:

                
            routes_container=wait.until(EC.presence_of_all_elements_located((By.XPATH,"//div[@class='clearfix bus-item']")))
            time.sleep(3)
                            
            for bus in routes_container:

                try:
                    bus_name=bus.find_element(By.XPATH,".//div[@class='travels lh-24 f-bold d-color']").text
                except:
                    bus_name="NA"
                
                try:
                    bus_type=bus.find_element(By.XPATH,".//div[@class='bus-type f-12 m-top-16 l-color evBus']").text
                except:
                    bus_type="NA"
                
                try:
                    departure_time=bus.find_element(By.XPATH,".//div[@class='dp-time f-19 d-color f-bold']").text
                except:
                    departure_time="NA"
                
                try:
                    duration=bus.find_element(By.XPATH,".//div[@class='dur l-color lh-24']").text
                except:
                    duration="NA"
                
                try:
                    arrival_time=bus.find_element(By.XPATH,".//div[@class='bp-time f-19 d-color disp-Inline']").text
                except:
                    arrival_time="NA"
                
                try:
                    ratings=bus.find_element(By.XPATH,".//div[@class='rating-sec lh-24']").text
                except:
                    ratings=0
                
                try:
                    price=bus.find_element(By.XPATH,".//span[@class='f-19 f-bold']").text
                except:
                    price=0
                
                try:
                    seats_availability=bus.find_element(By.XPATH,".//div[@class='column-eight w-15 fl']").text
                except:
                    seats_availability="NA"
                
                bus_details_text.append({'Route_link':link,'Bus_name':bus_name,'Bus_type':bus_type,'Dep_time':departure_time,'Duration':duration,'Arr_time':arrival_time,'Rating':ratings,'Price':price,'Seat_avail':seats_availability})
                
            
            return bus_details_text
        
        except:
            print('NA')
    try:  
        view_buses=driver.find_element(By.XPATH,'//div[@class="button"]')
        time.sleep(5)
        view_buses.click()
        time.sleep(5)
        bus_details()
        for _ in range(1,45):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    try:
        view_buses2=wait.until(EC.visibility_of_element_located((By.XPATH,'//div[contains(@class,"button") and contains(text(),"View Buses")]')))
        time.sleep(3)
        Action=ActionChains(driver)
        Action.move_to_element(view_buses2).perform()

        time.sleep(3)
        view_buses2.click()
        time.sleep(3)
        for _ in range(1,40):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)
        bus_details()

    except:
        bus_details()
        for _ in range(1,30):
            body = driver.find_element(By.TAG_NAME, 'body')
            body.send_keys(Keys.PAGE_DOWN)

    time.sleep(3)
    for _ in range(1,100):
        body = driver.find_element(By.TAG_NAME, 'body')
        body.send_keys(Keys.PAGE_DOWN)
    bus_details()





NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA
NA


In [76]:
JK_route=pd.DataFrame(route_jk)
JK_bus_details=pd.DataFrame(bus_details_text)

In [77]:
JK_bus_details.dropna()

,Route_link,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail
0,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,1.4,0,36 Seats available\n18 Window
1,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),16:30,10h 00m,02:30,1.0,699,40 Seats available\n20 Window
2,https://www.redbus.in/bus-tickets/jammu-to-sri...,Jamna Travels-Jammu,Non AC Seater (2+2),21:30,10h 30m,08:00,2.3,0,26 Seats available\n13 Window
3,https://www.redbus.in/bus-tickets/jammu-to-sri...,North Kashmir Tour and Travels,NON A/C Sleeper (2+2),17:15,10h 00m,03:15,1.0,699,40 Seats available\n20 Window
4,https://www.redbus.in/bus-tickets/jammu-to-sri...,New Pal Travels,NON A/C Seater / Sleeper (2+2),21:15,10h 00m,07:15,1.4,0,36 Seats available\n18 Window
...,...,...,...,...,...,...,...,...,...
449,https://www.redbus.in/bus-tickets/jammu-to-amr...,Neeraj travels,A/C Seater / Sleeper (2+1),15:40,04h 32m,20:12,2.5,599,25 Seats available\n1 Single
450,https://www.redbus.in/bus-tickets/jammu-to-amr...,Vijay Tour and Travels,Bharat Benz A/C Sleeper (2+1),16:30,04h 30m,21:00,2.2,0,30 Seats available\n20 Window
451,https://www.redbus.in/bus-tickets/ajmer-to-jammu,Vijay Tour and Travels,A/C Seater / Sleeper (2+1),15:00,17h 50m,08:50,3.2,0,14 Seats available\n1 Single
452,https://www.redbus.in/bus-tickets/ajmer-to-jammu,Vijay Tour and Travels,A/C Seater / Sleeper (2+1),15:00,17h 50m,08:50,3.2,0,14 Seats available\n1 Single


In [78]:
JK_bus=pd.merge(JK_route,JK_bus_details,on='Route_link')

In [79]:
JK_bus.to_csv('JK_bus_details.csv')

In [80]:
JK_bus['Price']=JK_bus['Price'].replace('',0)

In [81]:
JK_bus['Rating']=JK_bus['Rating'].replace('',0)

In [82]:
import pymysql
con=pymysql.connect(
    host="localhost",
    user="root",
    password="Omsairam_10",
    autocommit=True
)
mycursor=con.cursor()
mycursor

In [83]:
mycursor.execute("USE RedBus_project_new")
create_table_query="""CREATE TABLE IF NOT EXISTS JK_bus_details(id int auto_increment primary key,
                    Route_link varchar(250),
                    Route_name varchar(250),
                    Bus_name varchar(250),
                    Bus_type varchar(250),
                    Dep_time time,
                    Duration varchar(100),
                    Arr_time time,
                    Rating decimal(20,2),
                    Price float,
                    Seat_avail varchar(200));"""
mycursor.execute(create_table_query)

con.commit()

data="INSERT INTO JK_bus_details(Route_link,Route_name,Bus_name,Bus_type,Dep_time,Duration,Arr_time,Rating,Price,Seat_avail) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
for index,row in JK_bus.iterrows():
    mycursor.execute(data,(row['Route_link'],
                           row['Route_name'],
                           row['Bus_name'],
                           row['Bus_type'],
                           row['Dep_time'],
                           row['Duration'],
                           row['Arr_time'],
                           row['Rating'],
                           row['Price'],
                           row['Seat_avail']))
con.commit()



